In [17]:
import pandas as pd
pd.options.display.max_rows
import geocoder
from datetime import datetime
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

# 1  TRIPS & CENSUS DATA

# 1.1  TRIPS DATASET WORK

In [4]:
# Start importing the trip data from December
december = pd.read_csv('202212-citibike-tripdata.csv')
december.head(2)

C:\Users\javierheras\AppData\Local\Temp\ipykernel_5872\1875490524.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  december = pd.read_csv('202212-citibike-tripdata.csv')


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0EF496F342AD4E7E,classic_bike,2022-12-05 17:47:32,2022-12-05 18:13:10,DeKalb Ave & Hudson Ave,4513.06,Perry St & Bleecker St,5922.07,40.689909,-73.980552,40.735354,-74.004831,member
1,4B9482CBC3FAC46B,classic_bike,2022-12-08 19:08:05,2022-12-08 19:20:23,E 128 St & Madison Ave,7735.23,E 142 St & 3 Ave,7781.10,40.807518,-73.939027,40.812735,-73.923697,member


In [11]:
# we want to know the approximate number of bike stations in NY
december_stations = december.groupby(['start_station_id'])
december_ok = december_stations.first().reset_index()

In [13]:
december_st_coor = december_ok[['start_station_id','start_lat','start_lng']]
december_st_coor.head(2)

,start_station_id,start_lat,start_lng
0,2733.03,40.633448,-74.016587
1,2782.02,40.635679,-74.020005


In [49]:
#The objective, once we have the coordinates of the stations, is to know the Community District 
#in which they are located, for this we use the geocoder library that gives us the postal code in which they are located.
for x in range(0,len(december_st_coor['start_station_id'])):
    g = geocoder.osm([december_st_coor['start_lat'][x],december_st_coor['start_lng'][x]], method='reverse')
    december_st_coor['start_cd'][x]= g.postal

C:\Users\javierheras\AppData\Local\Temp\ipykernel_19964\3428398436.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enero1['start_cd'][x]= g.postal


In [53]:
#We analyze the stations that the library has not found, a total of 9 out of more than 3000.
december_st_coor['start_cd'].isnull().sum()

9

In [54]:
station_list = list(december_st_coor['start_station_id'])

In [55]:
zipcode_lits = list(december_st_coor['start_cd'])

In [56]:
stations_tuple = list(zip(station_list,zipcode_lits))

In [58]:
#We create a dictionary with the id of the stations and the postal code to which they belong, 
#so that we can use it in all the datasets and include this information in them.
stations_dic = dict(stations_tuple)

In [59]:
#We create two new columns with the zip codes to which the origin and destination stations belong.
december = december.assign(Zip_code_start=december.start_station_id.map(stations_dic))

In [60]:
december = december.assign(Zip_code_end=december.end_station_id.map(stations_dic))

In [64]:
december['Zip_code_start']= pd.to_numeric(december['Zip_code_start'])

In [65]:
december['Zip_code_end']= pd.to_numeric(december['Zip_code_end'])

In [66]:
december = december.dropna()

In [17]:
#We created a dictionary to link postal districts to community districts
Coincidence_data = {101: ["Battery Park City", [10278,10004,10005,10006,10007,10013,10038,10041,10280,10281,10282,10285]], 102: ["Greenwich Village & Soho",[10012,10003,10014]],103:["Chinatown & Lower East Side",[10002,10009]],
                    104:["Chelsea, Clinton",[10011,10001,10019]],105:["Midtown Business District",[10055,10111,10018,10020,10022,10036,10154]],106:["Murray Hill, Gramercy & Stuyvesant Town",[10010,10016,10017,10174]],
                    107:["Upper West Side & West Side",[10023,10024,10025,10069]],108:["Upper East Side",[10021,10028,10044,10065,10075,10128]],109:["Hamilton Heights, Manhattanville & West Harlem",[10027,10031,10115]],
                    110:["Central Harlem",[10026,10030,10039]],111:["East Harlem",[10029,10035,10037]],112:["Washington Heights, Inwood & Marble Hill",[10032,10033,10034,10040]],
                    
                    201:["Hunts Point",[10451,10454,10455]],202:["Longwood & Melrose",[10459,10474]],
                    203:["Belmont, Crotona Park East",[10456]],204:["Concourse, Highbridge & Mount Eden",[10452]],205:["Morris Heights, Fordham South & Mount Hope",[10453]],206:["Crotona Park East & East",[10460,10457,10460,10458]],
                    207:["Bedford Park, Fordham North & Norwood",[10468]],208:["Riverdale, Fieldston & Kingsbridge",[10463,10471]],209:["Castle Hill, Clason Point & Parkchester",[10742,10473]],
                    210:["Co-op City, Pelham Bay & Schuylerville",[10464,10465,10475]],211:["Pelham Parkway, Morris Park & Laconia",[10462,10461,10469]],212:["Wakefield, Williamsbridge & Woodlawn",[10467,10466,10470]],
                    
                    301:["Greenpoint & Williamsburg",[11206,11211,11222,11249]],302:["Brooklyn Heights & Fort Greene",[11201,11205,11217,11251]],303:["Bedford-Stuyvesant",[11216]],304:["Bushwick",[11237,11221]],
                    305:["East New York & Starrett City",[11207,11208,11239]],306:["Park Slope, Carroll Gardens & Red Hook",[11215,11231]],
                    307:["Sunset Park & Windsor Terrace ",[11220,11232]],308:["Crown Heights North & Prospect Heights",[11238,11213]],
                    309:["Crown Heights South, Prospect Lefferts & Wingate",[11225]],310:["Bay Ridge & Dyker Heights",[11209,11228]],
                    311:["Bensonhurst & Bath Beach",[11204,11214]],312:["Borough Park, Kensington & Ocean Parkway",[11219,11218]],313:["Brighton Beach & Coney Island",[11224]],
                    314:["Flatbush & Midwood",[11226,11230,11210]],315:["Sheepshead Bay, Gerritsen Beach & Homecrest",[11223,11229,11235]],
                    316:["Brownsville & Ocean Hill",[11233,11212]],317:["East Flatbush, Farragut & Rugby",[11203]],318:["Canarsie & Flatlands",[11236]],            
                    
                    
                    
                    401:["Astoria & Long Island City",[11102,11103,11105,11106]],402:["Sunnyside & Woodside",[11101,11377,11104]],403:["Jackson Heights & North Corona",[11370,11369,11372]],
                    404:["Elmhurst & South Corona",[11368,11373]],405:["Ridgewood, Glendale & Middle Village",[11378,11379,11385]],406:["Forest Hills & Rego Park",[11374,11375]],
                    407:["Sunset Park & Windsor Terrace",[11354,11355,11356,11357,11358,11359,11360]],408:["Briarwood, Fresh Meadows & Hillcrest",[11365,11367,11366,11432]],
                    409:["Richmond Hill & Woodhaven",[11415,11416,11418,11421]],410:["Howard Beach & Ozone Park",[11419,11414,11417,11420]],
                    411:["Bayside, Douglaston & Little Neck",[11364,11423]],412:["Jamaica, Hollis & St. Albans",[11435,11412,11423,11433,11434,11436]],
                    413:["Queens Village, Cambria Heights & Rosedale",[11427,11001,11005,11413,11004,11040,11411,11413,11422,11426,11428,11429,11430]],
                    414:["Far Rockaway, Breezy Point & Broad Channel",[11691,11694,11692,11697,11693]],
                    
                    501:["Port Richmond, Stapleton & Mariner's Harbor",[10301,10302,10303,10310]],502:["New Springville & South Beach",[10304,10305,10314]],503:["Tottenville, Great Kills & Annadale",[10306,10307,10308,10309,10312]]
                   }

In [ ]:
BR_names = {'Manhattan':[101,102,103,104,105,106,107,108,109,110,111,112],
            'Bronx':[201,202,203,204,205,206,207,208,209,210,211,212,],
           'Brooklyn':[301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318],
           'Queens':[401,402,403,404,405,406,407,408,409,410,411,412,413,414],
           'Staten Island':[501,502,503]}

In [18]:
def name_district_finder(x):
    for key, value in Coincidence_data.items():
        if x in value[1]:
            return value[0]

In [22]:
def code_district_finder(x):
    for key, value in Coincidence_data.items():
        if x in value[1]:
            return key

In [ ]:
def name_br_finder(x):
    for key, value in BR_names.items():
        if x in value:
            return key

In [71]:
#With the application of the above functions we can add to our dataset the information of 
#the code of the Community district and its name
december['start_district_name'] = december.apply(lambda row : name_district_finder(row['Zip_code_start']), axis=1)

In [73]:
december['end_district_name'] = december.apply(lambda row : name_district_finder(row['Zip_code_end']), axis=1)

In [75]:
december['start_district_code'] = december.apply(lambda row : code_district_finder(row['Zip_code_start']), axis=1)

In [77]:
december['end_district_code'] = december.apply(lambda row : code_district_finder(row['Zip_code_end']), axis=1)

In [ ]:
december = december.dropna()

In [ ]:
december['BR_name'] = december.apply(lambda row : name_br_finder(row['start_district_code']), axis=1)

In [79]:
def haversine(lat1, lon1, lat2, lon2):
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=(2*R*math.asin(math.sqrt(a)))*1000
    return distancia

In [ ]:
december['distance'] = december.apply(lambda row : haversine(row['start_lat'],row['start_lng'],row['end_lat'],row['end_lng']), axis=1)

In [ ]:
december = december.dropna()

In [80]:
december['started_at'] = pd.to_datetime(december['started_at'])
december['ended_at'] = pd.to_datetime(december['ended_at'])

In [83]:
#this function creates a column for the year, month and day of the date of each trip, 
#and also creates two more columns that reflect the seconds and minutes of each trip.
def dates(df):
    df['start_year']=df['started_at'].dt.year
    df['start_month']=df['started_at'].dt.month
    df['start_day']=df['started_at'].dt.day
    t = df['ended_at']-df['started_at']
    df['travel_time_sc'] = t.dt.seconds
    df['travel_time_minut'] = df['travel_time_sc']/60
    df['hours'] = df['started_at'].dt.hour
    df['day_name'] = df['started_at'].apply(lambda x : x.strftime('%A'))
    df['Month_name'] = df['started_at'].apply(lambda x : x.strftime('%B'))   
    
    return df

In [ ]:
december = dates(december)

In [120]:
#Applying this function to each dataset of the trips corresponding to each month, 
#we obtain the transformation obtained previously in the month of December, which we have explained step by step.

def comprehensive_dataset_work(df):
    df = df.assign(Zip_code_start=df.start_station_id.map(stations_dic))
    df = df.assign(Zip_code_end=df.end_station_id.map(stations_dic))
    
    
    df['Zip_code_start']= pd.to_numeric(df['Zip_code_start'])
    df['Zip_code_end']= pd.to_numeric(df['Zip_code_end'])
    
    df = df.dropna()
    
    
    df['start_district_name'] = df.apply(lambda row : name_district_finder(row['Zip_code_start']), axis=1)    
    df['end_district_name'] = df.apply(lambda row : name_district_finder(row['Zip_code_end']), axis=1)
    
    
    df['start_district_code'] = df.apply(lambda row : code_district_finder(row['Zip_code_start']), axis=1)    
    df['end_district_code'] = df.apply(lambda row : code_district_finder(row['Zip_code_end']), axis=1)
    
    df = df.dropna()
    
    df['BR_name'] = df.apply(lambda row : name_br_finder(row['start_district_code']), axis=1)
    
    df['distance'] = df.apply(lambda row : haversine(row['start_lat'],row['start_lng'],row['end_lat'],row['end_lng']), axis=1)
    
    df = df.dropna()
    
    df['started_at'] = pd.to_datetime(df['started_at'])
    df['ended_at'] = pd.to_datetime(df['ended_at'])
    
    
    df = dates(df)
    
    
    return df    

In [87]:
november = pd.read_csv('202211-citibike-tripdata.csv')

C:\Users\javierheras\AppData\Local\Temp\ipykernel_19964\2226983112.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  november = pd.read_csv('202211-citibike-tripdata.csv')


In [88]:
october = pd.read_csv('202210-citibike-tripdata.csv')

C:\Users\javierheras\AppData\Local\Temp\ipykernel_19964\680787243.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  october = pd.read_csv('202210-citibike-tripdata.csv')


In [89]:
september = pd.read_csv('202209-citibike-tripdata.csv')

C:\Users\javierheras\AppData\Local\Temp\ipykernel_19964\2179174460.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  september = pd.read_csv('202209-citibike-tripdata.csv')


In [90]:
august = pd.read_csv('202208-citibike-tripdata.csv')

C:\Users\javierheras\AppData\Local\Temp\ipykernel_19964\193552100.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  august = pd.read_csv('202208-citibike-tripdata.csv')


In [91]:
july = pd.read_csv('202207-citbike-tripdata.csv')

C:\Users\javierheras\AppData\Local\Temp\ipykernel_19964\522501241.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  july = pd.read_csv('202207-citbike-tripdata.csv')


In [92]:
june = pd.read_csv('202206-citbike-tripdata.csv')

C:\Users\javierheras\AppData\Local\Temp\ipykernel_19964\1346284269.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  june = pd.read_csv('202206-citbike-tripdata.csv')


In [93]:
may = pd.read_csv('202205-citibike-tripdata.csv')

C:\Users\javierheras\AppData\Local\Temp\ipykernel_19964\1503459682.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  may = pd.read_csv('202205-citibike-tripdata.csv')


In [94]:
april = pd.read_csv('202204-citibike-tripdata.csv')

C:\Users\javierheras\AppData\Local\Temp\ipykernel_19964\1949911740.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  april = pd.read_csv('202204-citibike-tripdata.csv')


In [95]:
march = pd.read_csv('202203-citibike-tripdata.csv')

C:\Users\javierheras\AppData\Local\Temp\ipykernel_19964\310835775.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  march = pd.read_csv('202203-citibike-tripdata.csv')


In [96]:
february = pd.read_csv('202202-citibike-tripdata.csv')

C:\Users\javierheras\AppData\Local\Temp\ipykernel_19964\4165505551.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  february = pd.read_csv('202202-citibike-tripdata.csv')


In [97]:
january = pd.read_csv('202201-citibike-tripdata.csv')

C:\Users\javierheras\AppData\Local\Temp\ipykernel_19964\1796573794.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  january = pd.read_csv('202201-citibike-tripdata.csv')


In [121]:
november = comprehensive_dataset_work(november)

In [122]:
october = comprehensive_dataset_work(october)

In [123]:
september = comprehensive_dataset_work(september)

In [124]:
august = comprehensive_dataset_work(august)

In [125]:
july = comprehensive_dataset_work(july)

In [126]:
june = comprehensive_dataset_work(june)

In [127]:
may = comprehensive_dataset_work(may)

In [128]:
april = comprehensive_dataset_work(april)

In [129]:
march = comprehensive_dataset_work(march)

In [130]:
february = comprehensive_dataset_work(february)

In [131]:
january = comprehensive_dataset_work(january)

In [132]:
year_dataset = pd.concat([january,february,march,april,may,june,july,august,september,october,november,december], axis = 0)

In [140]:
year_dataset.to_csv('year_dataset.csv')

In [ ]:
#We will differentiate by type of person who uses the bicycle, we are interested in those who use it continuously, 
#it is to those who will be directed our advertising, so we will create a dataset for the latter.
year_dataset_member = year_dataset[year_dataset['member_casual']=='member']

In [ ]:
year_dataset_casual = year_dataset[year_dataset['member_casual']=='casual']

In [ ]:
year_dataset_member=year_dataset_member.drop(['Unnamed: 0.1','Unnamed: 0'],axis=1)
year_dataset_casual=year_dataset_casual.drop(['Unnamed: 0.1','Unnamed: 0'],axis=1)

In [ ]:
year_dataset_member.to_csv('year_dataset_member.csv')

In [ ]:
year_dataset_casual.to_csv('year_dataset_casual.csv')

In [ ]:
#Now we are going to eliminate outliners or trip errors, trips with travel time of less than 30 seconds and 
#more than two hours are eliminated.
year_dataset_member.drop(year_dataset_member[year_dataset_member['travel_time_sc'] >7200].index,inplace=True)

In [ ]:
year_dataset_member.drop(year_dataset_member[year_dataset_member['travel_time_sc'] <30].index,inplace=True)

In [ ]:
#Once we have the dataset with all the information we need for the analysis, 
#we make a sample of 14000000 million trips, tableau works less efficiently with datasets of between 10 and 15 million lines, 
#but we need a sufficiently representative dataset for our analysis.
year_dataset_member_tableau = year_dataset_member.sample(14000000)

In [ ]:
year_dataset_member_tableau =  pd.read_csv('year_dataset_member_tableau.csv')

# 1.2  CENSUS DATA

In [20]:
#we import the census data that appears in the web https://data.census.gov/
census_cd_percentage = pd.read_csv('census_cd_completo.csv')

In [21]:
#remove the data which is not a percentage, we only want the percentages to show its in Tableau
census_cd_percentage = census_cd_percentage.drop([
 
 'New York!!Estimate',
 
 'NYC-Bronx Community District 8--Riverdale, Fieldston & Kingsbridge PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 12--Wakefield, Williamsbridge & Woodlawn PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 10--Co-op City, Pelham Bay & Schuylerville PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 11--Pelham Parkway, Morris Park & Laconia PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 3 & 6--Belmont, Crotona Park East & East Tremont PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 7--Bedford Park, Fordham North & Norwood PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 5--Morris Heights, Fordham South & Mount Hope PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 4--Concourse, Highbridge & Mount Eden PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 9--Castle Hill, Clason Point & Parkchester PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 1 & 2--Hunts Point, Longwood & Melrose PUMA; New York!!Estimate',
 
 'NYC-Manhattan Community District 12--Washington Heights, Inwood & Marble Hill PUMA; New York!!Estimate',
 
 'NYC-Manhattan Community District 9--Hamilton Heights, Manhattanville & West Harlem PUMA; New York!!Estimate',
 
 'NYC-Manhattan Community District 10--Central Harlem PUMA, New York!!Estimate',
 
 'NYC-Manhattan Community District 11--East Harlem PUMA, New York!!Estimate',
 
 'NYC-Manhattan Community District 8--Upper East Side PUMA, New York!!Estimate',
 
 'NYC-Manhattan Community District 7--Upper West Side & West Side PUMA, New York!!Estimate',
 
 'NYC-Manhattan Community District 4 & 5--Chelsea, Clinton & Midtown Business District PUMA; New York!!Estimate',
 
 'NYC-Manhattan Community District 6--Murray Hill, Gramercy & Stuyvesant Town PUMA; New York!!Estimate',
 
 'NYC-Manhattan Community District 3--Chinatown & Lower East Side PUMA, New York!!Estimate',

 'NYC-Manhattan Community District 1 & 2--Battery Park City, Greenwich Village & Soho PUMA; New York!!Estimate',
 
 'NYC-Staten Island Community District 3--Tottenville, Great Kills & Annadale PUMA; New York!!Estimate',

 'NYC-Staten Island Community District 2--New Springville & South Beach PUMA, New York!!Estimate',
 
 "NYC-Staten Island Community District 1--Port Richmond, Stapleton & Mariner's Harbor PUMA; New York!!Estimate",
 
 'NYC-Brooklyn Community District 1--Greenpoint & Williamsburg PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 4--Bushwick PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 3--Bedford-Stuyvesant PUMA, New York!!Estimate',

 'NYC-Brooklyn Community District 2--Brooklyn Heights & Fort Greene PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 6--Park Slope, Carroll Gardens & Red Hook PUMA; New York!!Estimate',
 
 'NYC-Brooklyn Community District 8--Crown Heights North & Prospect Heights PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 16--Brownsville & Ocean Hill PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 5--East New York & Starrett City PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 18--Canarsie & Flatlands PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 17--East Flatbush, Farragut & Rugby PUMA; New York!!Estimate',

 'NYC-Brooklyn Community District 9--Crown Heights South, Prospect Lefferts & Wingate PUMA; New York!!Estimate',

 'NYC-Brooklyn Community District 7--Sunset Park & Windsor Terrace PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 10--Bay Ridge & Dyker Heights PUMA, New York!!Estimate',

 'NYC-Brooklyn Community District 12--Borough Park, Kensington & Ocean Parkway PUMA; New York!!Estimate',
 
 'NYC-Brooklyn Community District 14--Flatbush & Midwood PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 15--Sheepshead Bay, Gerritsen Beach & Homecrest PUMA; New York!!Estimate',
 
 'NYC-Brooklyn Community District 11--Bensonhurst & Bath Beach PUMA, New York!!Estimate',

 'NYC-Brooklyn Community District 13--Brighton Beach & Coney Island PUMA, New York!!Estimate',
 
 'NYC-Queens Community District 1--Astoria & Long Island City PUMA, New York!!Estimate',

 'NYC-Queens Community District 3--Jackson Heights & North Corona PUMA, New York!!Estimate',

 'NYC-Queens Community District 7--Flushing, Murray Hill & Whitestone PUMA; New York!!Estimate',
 
 'NYC-Queens Community District 11--Bayside, Douglaston & Little Neck PUMA; New York!!Estimate',
 
 'NYC-Queens Community District 13--Queens Village, Cambria Heights & Rosedale PUMA; New York!!Estimate',

 'NYC-Queens Community District 8--Briarwood, Fresh Meadows & Hillcrest PUMA; New York!!Estimate',
 
 'NYC-Queens Community District 4--Elmhurst & South Corona PUMA, New York!!Estimate',

 'NYC-Queens Community District 6--Forest Hills & Rego Park PUMA, New York!!Estimate',

 'NYC-Queens Community District 2--Sunnyside & Woodside PUMA, New York!!Estimate',
 
 'NYC-Queens Community District 5--Ridgewood, Glendale & Middle Village PUMA; New York!!Estimate',
 
 'NYC-Queens Community District 9--Richmond Hill & Woodhaven PUMA, New York!!Estimate',
 
 'NYC-Queens Community District 12--Jamaica, Hollis & St. Albans PUMA; New York!!Estimate',

 'NYC-Queens Community District 10--Howard Beach & Ozone Park PUMA, New York!!Estimate',

 'NYC-Queens Community District 14--Far Rockaway, Breezy Point & Broad Channel PUMA; New York!!Estimate'], axis=1)

In [22]:
#remove all the data less the gender, age and etnic data
census_cd_percentage = census_cd_percentage.drop([0,1,4,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,
                                                 75,76,77,78,79,87,88,89,90,91,92,93],axis=0)

In [23]:
census_cd_percentage = census_cd_percentage.set_index('Label (Grouping)')

In [24]:
census_cd_percentage = census_cd_percentage.transpose()

In [25]:
census_cd_percentage = census_cd_percentage.drop('New York!!Percent',axis=0)

In [26]:
census_cd_percentage =  census_cd_percentage.replace('%','',regex=True)  

In [27]:
census_cd_percentage =  census_cd_percentage.astype(float)

In [28]:
repe = census_cd_percentage.iloc[[4,9,16,19]]

In [29]:
repe =(repe/2)

In [30]:
census_cd_percentage = census_cd_percentage.drop(["NYC-Bronx Community District 3 & 6--Belmont, Crotona Park East & East Tremont PUMA; New York!!Percent",
                            "NYC-Bronx Community District 1 & 2--Hunts Point, Longwood & Melrose PUMA; New York!!Percent",
                            "NYC-Manhattan Community District 4 & 5--Chelsea, Clinton & Midtown Business District PUMA; New York!!Percent"
                            ,"NYC-Manhattan Community District 1 & 2--Battery Park City, Greenwich Village & Soho PUMA; New York!!Percent"],axis=0)

In [31]:
census_cd_percentage = census_cd_percentage.append([repe]*2)

C:\Users\javierheras\AppData\Local\Temp\ipykernel_5872\1580879092.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  census_cd_percentage = census_cd_percentage.append([repe]*2)


In [32]:
CD_code = [208,212,210,211,207,205,204,209,112,109,110,111,108,107
           ,106,103,503,502,501,301,304,303,302,306,308,316,305,318,317,
           309,307,310,312,314,315,311,313,401,403,407,411,413,408,404,406,402,405,409,412,410,414,203,201,104,101,206,202,105,102]

In [33]:
BC = ['Bronx','Bronx','Bronx','Bronx','Bronx','Bronx','Bronx','Bronx','Manhattan','Manhattan',
      'Manhattan','Manhattan','Manhattan','Manhattan','Manhattan','Manhattan','Staten Island','Staten Island','Staten Island',
     'Brooklyn','Brooklyn','Brooklyn','Brooklyn','Brooklyn','Brooklyn','Brooklyn','Brooklyn','Brooklyn','Brooklyn','Brooklyn',
      'Brooklyn','Brooklyn','Brooklyn','Brooklyn','Brooklyn','Brooklyn','Brooklyn','Queens','Queens','Queens','Queens','Queens','Queens',
      'Queens','Queens','Queens','Queens','Queens','Queens','Queens','Queens','Bronx','Bronx','Manhattan','Manhattan','Bronx','Bronx','Manhattan','Manhattan']

In [34]:
census_cd_percentage['BD_name'] = BC

In [35]:
census_cd_percentage['CD_code'] = CD_code

In [ ]:
census_cd_percentage.to_csv('census_cd_percentage_tableau.csv')

In [ ]:
census_cd2_percentage = pd.read_csv('census_completo.csv')

In [ ]:
census_cd2_percentage = census_cd2_percentage.drop([
 
 'New York!!Estimate',
 
 'NYC-Bronx Community District 8--Riverdale, Fieldston & Kingsbridge PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 12--Wakefield, Williamsbridge & Woodlawn PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 10--Co-op City, Pelham Bay & Schuylerville PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 11--Pelham Parkway, Morris Park & Laconia PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 3 & 6--Belmont, Crotona Park East & East Tremont PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 7--Bedford Park, Fordham North & Norwood PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 5--Morris Heights, Fordham South & Mount Hope PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 4--Concourse, Highbridge & Mount Eden PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 9--Castle Hill, Clason Point & Parkchester PUMA; New York!!Estimate',
 
 'NYC-Bronx Community District 1 & 2--Hunts Point, Longwood & Melrose PUMA; New York!!Estimate',
 
 'NYC-Manhattan Community District 12--Washington Heights, Inwood & Marble Hill PUMA; New York!!Estimate',
 
 'NYC-Manhattan Community District 9--Hamilton Heights, Manhattanville & West Harlem PUMA; New York!!Estimate',
 
 'NYC-Manhattan Community District 10--Central Harlem PUMA, New York!!Estimate',
 
 'NYC-Manhattan Community District 11--East Harlem PUMA, New York!!Estimate',
 
 'NYC-Manhattan Community District 8--Upper East Side PUMA, New York!!Estimate',
 
 'NYC-Manhattan Community District 7--Upper West Side & West Side PUMA, New York!!Estimate',
 
 'NYC-Manhattan Community District 4 & 5--Chelsea, Clinton & Midtown Business District PUMA; New York!!Estimate',
 
 'NYC-Manhattan Community District 6--Murray Hill, Gramercy & Stuyvesant Town PUMA; New York!!Estimate',
 
 'NYC-Manhattan Community District 3--Chinatown & Lower East Side PUMA, New York!!Estimate',

 'NYC-Manhattan Community District 1 & 2--Battery Park City, Greenwich Village & Soho PUMA; New York!!Estimate',
 
 'NYC-Staten Island Community District 3--Tottenville, Great Kills & Annadale PUMA; New York!!Estimate',

 'NYC-Staten Island Community District 2--New Springville & South Beach PUMA, New York!!Estimate',
 
 "NYC-Staten Island Community District 1--Port Richmond, Stapleton & Mariner's Harbor PUMA; New York!!Estimate",
 
 'NYC-Brooklyn Community District 1--Greenpoint & Williamsburg PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 4--Bushwick PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 3--Bedford-Stuyvesant PUMA, New York!!Estimate',

 'NYC-Brooklyn Community District 2--Brooklyn Heights & Fort Greene PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 6--Park Slope, Carroll Gardens & Red Hook PUMA; New York!!Estimate',
 
 'NYC-Brooklyn Community District 8--Crown Heights North & Prospect Heights PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 16--Brownsville & Ocean Hill PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 5--East New York & Starrett City PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 18--Canarsie & Flatlands PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 17--East Flatbush, Farragut & Rugby PUMA; New York!!Estimate',

 'NYC-Brooklyn Community District 9--Crown Heights South, Prospect Lefferts & Wingate PUMA; New York!!Estimate',

 'NYC-Brooklyn Community District 7--Sunset Park & Windsor Terrace PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 10--Bay Ridge & Dyker Heights PUMA, New York!!Estimate',

 'NYC-Brooklyn Community District 12--Borough Park, Kensington & Ocean Parkway PUMA; New York!!Estimate',
 
 'NYC-Brooklyn Community District 14--Flatbush & Midwood PUMA, New York!!Estimate',
 
 'NYC-Brooklyn Community District 15--Sheepshead Bay, Gerritsen Beach & Homecrest PUMA; New York!!Estimate',
 
 'NYC-Brooklyn Community District 11--Bensonhurst & Bath Beach PUMA, New York!!Estimate',

 'NYC-Brooklyn Community District 13--Brighton Beach & Coney Island PUMA, New York!!Estimate',
 
 'NYC-Queens Community District 1--Astoria & Long Island City PUMA, New York!!Estimate',

 'NYC-Queens Community District 3--Jackson Heights & North Corona PUMA, New York!!Estimate',

 'NYC-Queens Community District 7--Flushing, Murray Hill & Whitestone PUMA; New York!!Estimate',
 
 'NYC-Queens Community District 11--Bayside, Douglaston & Little Neck PUMA; New York!!Estimate',
 
 'NYC-Queens Community District 13--Queens Village, Cambria Heights & Rosedale PUMA; New York!!Estimate',

 'NYC-Queens Community District 8--Briarwood, Fresh Meadows & Hillcrest PUMA; New York!!Estimate',
 
 'NYC-Queens Community District 4--Elmhurst & South Corona PUMA, New York!!Estimate',

 'NYC-Queens Community District 6--Forest Hills & Rego Park PUMA, New York!!Estimate',

 'NYC-Queens Community District 2--Sunnyside & Woodside PUMA, New York!!Estimate',
 
 'NYC-Queens Community District 5--Ridgewood, Glendale & Middle Village PUMA; New York!!Estimate',
 
 'NYC-Queens Community District 9--Richmond Hill & Woodhaven PUMA, New York!!Estimate',
 
 'NYC-Queens Community District 12--Jamaica, Hollis & St. Albans PUMA; New York!!Estimate',

 'NYC-Queens Community District 10--Howard Beach & Ozone Park PUMA, New York!!Estimate',

 'NYC-Queens Community District 14--Far Rockaway, Breezy Point & Broad Channel PUMA; New York!!Estimate'], axis=1)

In [ ]:
census_cd2_percentage = census_cd2_percentage.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,24,25,26,27,28,29,30,31,32,33,34,35
                              ,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,63,64,65,66,76,77,78,79,80,81,82,83,84,
                              85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,
                              117,118,119,120,121,122,123,124,125,126,127,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,
                              159,160,161,162,163,164,165,166,167,168,169]
                              , axis=0)

In [ ]:
census_cd2_percentage = census_cd2_percentage.set_index('Label (Grouping)')

In [ ]:
census_cd2_percentage = census_cd2_percentage.transpose()

In [ ]:
census_cd2_percentage = census_cd2_percentage.drop('New York!!Percent',axis=0)

In [ ]:
census_cd2_percentage =  census_cd2_percentage.replace('%','',regex=True)   

In [ ]:
census_cd2_percentage =  census_cd2_percentage.astype(float)

In [ ]:
repe2 = census_cd2_percentage.iloc[[4,9,16,19]]

In [ ]:
repe2 =(repe2/2)

In [ ]:
census_cd2_percentage = census_cd2_percentage.drop(["NYC-Bronx Community District 3 & 6--Belmont, Crotona Park East & East Tremont PUMA; New York!!Percent",
                            "NYC-Bronx Community District 1 & 2--Hunts Point, Longwood & Melrose PUMA; New York!!Percent",
                            "NYC-Manhattan Community District 4 & 5--Chelsea, Clinton & Midtown Business District PUMA; New York!!Percent"
                            ,"NYC-Manhattan Community District 1 & 2--Battery Park City, Greenwich Village & Soho PUMA; New York!!Percent"],axis=0)

In [ ]:
census_cd2_percentage = census_cd2_percentage.append([repe2]*2)

In [ ]:
census_cd2_percentage['BD_name'] = BC

In [ ]:
census_cd2_percentage['CD_code'] = CD_code

In [ ]:
census_cd2_percentage.to_csv('census_cd2_percentage_tableau.csv')

In [ ]:
census_cd_percentage_total = census_cd_percentage.merge(census_cd2_percentage,on = 'CD_code',how='inner')

In [ ]:
census_cd_percentage_total.to_csv('census_cd_percentage_total.csv')

# 2º  CLUSTERING

In [ ]:
year_dataset_member_tableau =  pd.read_csv('year_dataset_member_tableau.csv')

In [ ]:
cluster_dataset = year_dataset_member_tableau.drop(['Unnamed: 0'],axis=1)

In [ ]:
cluster_dataset['bike_type'] = cluster_dataset['rideable_type'].replace({'classic_bike':0,'electric_bike':2})

In [ ]:
cluster_dataset['weekend'] = cluster_dataset['day_name'].replace({'Monday':0,'Tuesday':0,'Wednesday':0,'Thursday':0,
                                                                  'Friday':0,'Saturday':5,'Sunday':5})

In [ ]:
cluster_dataset['morning-evening'] = cluster_dataset['hours'].replace({0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:2,9:2,
                                                                      10:0,11:0,12:0,13:0,14:0,15:0,16:0,17:2,18:2,19:2,20:0,21:0,22:0,23:0,24:0})

In [ ]:
cluster_dataset_number = cluster_dataset.select_dtypes(include=np.number)

In [ ]:
cluster_dataset_number = cluster_dataset_number.drop(['start_district_code','end_district_code','start_day','start_lat','start_lng','end_lat','end_lng','Zip_code_start',
                                                      'Zip_code_end','start_year','travel_time_minut','hours','start_day','start_month','hours'],axis=1)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(cluster_dataset_number)

In [ ]:
data_scaled = scaler.transform(cluster_dataset_number)

In [ ]:
data_scaled = pd.DataFrame(data_scaled)

In [ ]:
model = KMeans()

In [ ]:
visualizer = KElbowVisualizer(model, k=(2,20))
visualizer.fit(data_scaled)
visualizer.poof()

In [ ]:
model = KMeans(n_clusters = 5)

In [ ]:
model.fit(data_scaled)

In [ ]:
clusters = model.predict(data_scaled)

In [ ]:
cluster_dataset["clusters"] = clusters

In [ ]:
cluster_dataset = cluster_dataset.drop(['bike_type','weekend','morning-evening'],axis=1)

In [ ]:
cluster_dataset.to_csv('year_dataset_member_tableau.csv')